In [ ]:

pip install tensorflow==2.15.*


In [ ]:
# import libraries
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Load data
train_df = pd.read_csv("train-data.tsv", sep="\t", names=["label", "message"])
test_df = pd.read_csv("valid-data.tsv", sep="\t", names=["label", "message"])

# Encode labels
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Tokenizer
max_vocab = 10000
max_len = 100
vectorize_layer = layers.TextVectorization(
    max_tokens=max_vocab,
    output_mode='int',
    output_sequence_length=max_len
)
vectorize_layer.adapt(train_df['message'])

# Prepare datasets
train_X = vectorize_layer(train_df['message'])
test_X = vectorize_layer(test_df['message'])
train_y = train_df['label'].values
test_y = test_df['label'].values

# Model: Embedding + Bidirectional LSTM
model = tf.keras.Sequential([
    layers.Embedding(max_vocab, 64),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(train_X, train_y, epochs=5, validation_split=0.2, batch_size=32)

# Prediction function
def predict_message(pred_text):
    vec = vectorize_layer([pred_text])
    prob = model.predict(vec, verbose=0)[0][0]
    label = "spam" if prob > 0.5 else "ham"
    return [float(prob), label]



In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
